In [1]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle fastai py7zr scikit-learn

In [2]:
from fastkaggle import *
import gc

In [3]:
# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not iskaggle and not cred_path.exists():
    creds = ''
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [4]:
comp = 'kkbox-music-recommendation-challenge'

path = setup_comp(comp, install='fastai')

In [5]:
path

Path('kkbox-music-recommendation-challenge')

In [6]:
from fastai.tabular.all import *
from fastai.collab import *

path.ls()

(#12) [Path('kkbox-music-recommendation-challenge/songs.csv'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv'),Path('kkbox-music-recommendation-challenge/train.csv.7z'),Path('kkbox-music-recommendation-challenge/members.csv.7z'),Path('kkbox-music-recommendation-challenge/sample_submission.csv'),Path('kkbox-music-recommendation-challenge/songs.csv.7z'),Path('kkbox-music-recommendation-challenge/members.csv'),Path('kkbox-music-recommendation-challenge/test.csv.7z'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv.7z'),Path('kkbox-music-recommendation-challenge/test.csv')...]

In [7]:
def is_extracted(file):
    return os.path.isfile(os.path.splitext(file)[0])

for file in path.ls(file_exts=".7z"):
    if not is_extracted(file):
        !py7zr x {str(file)} {str(path)}

In [8]:
path.ls(file_exts=".csv")

(#6) [Path('kkbox-music-recommendation-challenge/songs.csv'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv'),Path('kkbox-music-recommendation-challenge/sample_submission.csv'),Path('kkbox-music-recommendation-challenge/members.csv'),Path('kkbox-music-recommendation-challenge/test.csv'),Path('kkbox-music-recommendation-challenge/train.csv')]

In [9]:
songs = pd.read_csv(path/'train.csv', usecols=('msno', 'song_id', 'target'))

songs.head()

,msno,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


In [10]:
dls = CollabDataLoaders.from_df(songs, bs=512)
dls.show_batch()

,msno,song_id,target
0,R0CZkLuknweXo1lJ0vLDa18T0LuE7JEd+hgHpIMp5gQ=,pANXY+V/UYLXn+N/p/2GSj1bpeDiA5ValdYZCuNaOpY=,0
1,/0J5zoDaegQnAiBesjT85RAlB1C3AXgk6Uq0XSVtTng=,KkI9HqWZDzl/H+DwiWd5VwIaz3xCX38tpxDcrCL6mMY=,1
2,U/dAeSTSlnn2qCFONSv2XQ8W/t6DhfFFi9NSmHMlJwY=,podlycp3c6tOEDw7q8iQBVgvFjiMpkWWXTlXljGdbTY=,1
3,dRdSeEhTmUGx6lY71l+DOoBGRHM6myGJcGH+TnPmIqw=,Dzok/PPggjis8HettESjDLTC5OrnVxbkKxKQiYERPbo=,1
4,bxKJGEBfmhGl0uuNiNDakisbmfUKYFF55+aYhHyJmz8=,nnGh305rOmte1K/4D+qsL1LyY6P+jvDlzATIqwbCa/0=,1
5,jlWDpfUS+c+uBws5fGLUT7xRsAZH9rbzryWFqlrMVa8=,Ujt3R5hAe0Pix2mURuC+CO6aazhCvEU2lByPzy3E90k=,1
6,42OKcCYZo1ble/lpsBvSVDa7C31cChi1vetumlbVaMo=,GSzU4317vDd0bFvJKl/eBwZYQk6/SvGwzIcjw0XQjWY=,1
7,KaS2L8fC67G256UZ4xnzmUV7OIXU5kBeOEFyO9VFz6g=,DDNkiV58HqHHhAzIZh37mLE7qXzjXEPN1RwzLwtJSrY=,0
8,V8TXYEjy+JjOmetQH8cniw6HT2uk1FXNXxrnZ7YFo0g=,4v26O4A8AKu3NH132FO28/88IG8DwWT15vskmyV+UNo=,1
9,sdd7xO/MBV1sVi4G9yAEPaq2lOPSHtwBRpcr3mf3ABg=,n+FLlTL+LrNTa6xpoGkESEzlUMb/hPEM1nLq2U6fEp0=,1


In [11]:
from sklearn.metrics import roc_auc_score

def roc_metric(pred, target):
    return roc_auc_score(to_np(target.squeeze()), to_np(torch.clip(pred,0,1)))

learn = collab_learner(dls, y_range=(-0.1, 1.1), metrics=roc_metric)

In [12]:
learn.fit_one_cycle(1, 1e-1, wd=0.1)

epoch,train_loss,valid_loss,roc_metric,time
0,0.211044,0.217369,0.720092,02:07


In [13]:
# torch.save(learn.model.state_dict(), '01.pth')
# learn.model.load_state_dict(torch.load('01.pth'))
# learn.model.load_state_dict(torch.load('01.pth', map_location=torch.device('cpu')))

In [14]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(30756, 50)
  (i_weight): Embedding(359967, 50)
  (u_bias): Embedding(30756, 1)
  (i_bias): Embedding(359967, 1)
)

In [17]:
def merge_data(songs):
    song_data_0 = pd.read_csv(path/'songs.csv', usecols=('song_id', 'genre_ids', 'artist_name'))
    with_song_data = pd.merge(songs, song_data_0, how='left', on='song_id')
    del song_data_0
    del songs
    gc.collect()
    
    members = pd.read_csv(path/'members.csv', usecols=('msno', 'city', 'bd', 'gender'))
    with_members = pd.merge(with_song_data, members, how='left', on='msno')
    del with_song_data
    del members
    gc.collect()
    
    song_bias_df = pd.DataFrame({
        'song_id': learn.dls.classes['song_id'],
        'song_bias': to_np(learn.model.i_bias.weight.squeeze()),
    })
    song_weights_df = pd.DataFrame([
        {
            'song_id': song_id,
            **{f'song_weight_{i+1}': weight for i, weight in enumerate(weights)}
        }
        for song_id, weights in zip(learn.dls.classes['song_id'], learn.model.i_weight.weight.tolist())
    ])
    song_embeddings_df = pd.merge(song_bias_df, song_weights_df, on='song_id', how='left')

    del song_bias_df
    del song_weights_df
    gc.collect()

    with_song_embeddings = pd.merge(with_members, song_embeddings_df, how='left', on='song_id')
    del with_members
    del song_embeddings_df
    gc.collect()
    
    user_bias_df = pd.DataFrame({
        'msno': learn.dls.classes['msno'],
        'user_bias': to_np(learn.model.u_bias.weight.squeeze()),
    })
    user_weights_df = pd.DataFrame([
        {
            'msno': msno,
            **{f'user_weight_{i+1}': weight for i, weight in enumerate(weights)}
        }
        for msno, weights in zip(learn.dls.classes['msno'], learn.model.u_weight.weight.tolist())
    ])
    user_embeddings_df = pd.merge(user_bias_df, user_weights_df, on='msno', how='left')

    del user_bias_df
    del user_weights_df
    gc.collect()

    
    with_member_embeddings = pd.merge(with_song_embeddings, user_embeddings_df, how='left', on='msno')
    del with_song_embeddings
    del user_embeddings_df
    gc.collect()
    
    all_data = with_member_embeddings
    
    gc.collect()

    return all_data

all_data = merge_data(songs)

gc.collect()

all_data.head()

,msno,song_id,target,genre_ids,artist_name,city,bd,gender,song_bias,song_weight_1,...,user_weight_41,user_weight_42,user_weight_43,user_weight_44,user_weight_45,user_weight_46,user_weight_47,user_weight_48,user_weight_49,user_weight_50
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1,359,Bastille,1,0,NaN,-1.338013e-01,-6.649403e-03,...,-0.333518,-0.055322,0.183000,-0.235900,-0.074849,-0.025590,0.196428,0.336795,-0.082098,-0.002582
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1,1259,Various Artists,13,24,female,1.334380e-13,4.851794e-14,...,-0.027692,-0.094816,-0.098739,0.018662,0.065773,0.135241,0.059461,-0.003568,-0.010334,-0.044914
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1,1259,Nas,13,24,female,-1.515627e-02,-5.370787e-03,...,-0.027692,-0.094816,-0.098739,0.018662,0.065773,0.135241,0.059461,-0.003568,-0.010334,-0.044914
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1,1019,Soundway,13,24,female,-8.158122e-29,8.332299e-28,...,-0.027692,-0.094816,-0.098739,0.018662,0.065773,0.135241,0.059461,-0.003568,-0.010334,-0.044914
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1,1011,Brett Young,1,0,NaN,-3.595748e-01,2.327123e-02,...,-0.333518,-0.055322,0.183000,-0.235900,-0.074849,-0.025590,0.196428,0.336795,-0.082098,-0.002582


In [18]:
dep = 'target'
cont_names,cat_names = cont_cat_split(all_data, dep_var=dep)

In [20]:
def tabularify(data, is_train=True):
    y_names = None;
    splits = None;
    
    if is_train:
        y_names = dep;
        splits = RandomSplitter(valid_pct=0.2)(range_of(data));
        
    return TabularPandas(data,
                         procs=[Categorify, FillMissing],
                         cat_names=cat_names,
                         cont_names=cont_names,
                         y_names=y_names,
                         splits=splits)

to = tabularify(all_data)

X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_valid, y_valid = to.valid.xs, to.valid.ys.values.ravel()

del all_data
del to

gc.collect()

1322

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef

rf = RandomForestClassifier(100, min_samples_leaf=50)
rf.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=50)

In [24]:
del X_train
del y_train
gc.collect()

3423

In [25]:
roc_auc_score(y_valid, rf.predict(X_valid))

0.6880278085672232

In [26]:
del X_valid
del y_valid
gc.collect()

test_df = pd.read_csv(path/'test.csv', usecols=('msno', 'song_id', 'id'))

test_data = merge_data(test_df)

test_data.head()

,id,msno,song_id,genre_ids,artist_name,city,bd,gender,song_bias,song_weight_1,...,user_weight_41,user_weight_42,user_weight_43,user_weight_44,user_weight_45,user_weight_46,user_weight_47,user_weight_48,user_weight_49,user_weight_50
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,458,梁文音 (Rachel Liang),1,0,NaN,-0.162508,-0.038344,...,0.064754,-0.062669,-0.122783,0.038229,0.009236,-0.067023,-0.075687,0.040094,-0.029373,0.245957
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,465,林俊傑 (JJ Lin),1,0,NaN,0.282866,-0.138561,...,0.064754,-0.062669,-0.122783,0.038229,0.009236,-0.067023,-0.075687,0.040094,-0.029373,0.245957
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,2022,Yu Takahashi (高橋優),1,0,NaN,0.073593,0.037833,...,-0.042250,0.003510,-0.008908,0.012203,-0.003932,0.013958,0.016745,0.011442,0.074602,-0.044933
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,465,U2,3,30,male,0.029245,0.004537,...,-0.052388,0.233027,-0.038061,-0.037235,0.029004,-0.001611,-0.025451,-0.026487,0.011318,0.125826
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,873,Yoga Mr Sound,3,30,male,0.051091,-0.013534,...,-0.052388,0.233027,-0.038061,-0.037235,0.029004,-0.001611,-0.025451,-0.026487,0.011318,0.125826


In [27]:
to_test = tabularify(test_data, is_train=False)
X_test = to_test.train.xs

del to_test
gc.collect()

/usr/local/lib/python3.9/dist-packages/fastai/tabular/core.py:314: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  to.loc[:,n+'_na'] = missing[n]
/usr/local/lib/python3.9/dist-packages/fastai/tabular/core.py:314: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  to.loc[:,n+'_na'] = missing[n]
/usr/local/lib/python3.9/dist-packages/fastai/tabular/core.py:314: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at 

465

In [32]:
new_X_test = X_test.copy()

In [42]:
pattern = r'weight_\d+_na'
new_X_test.drop(columns=[col for col in new_X_test.columns if re.search(pattern, col)], inplace=True)

In [44]:
del new_X_test['user_bias_na']

In [45]:
output = rf.predict(new_X_test)

del new_X_test
gc.collect()

test_df = pd.read_csv(path/'test.csv', usecols=('msno', 'song_id', 'id'))
test_df['target'] = np.clip(output, 0, 1)

In [46]:
sub_df = test_df[['id','target']]
sub_df.to_csv(f'submission.csv', index=False)

In [47]:
!head submission.csv

id,target
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0


In [50]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('submission.csv', 'kkbox-recommendation-6', comp)

100%|██████████| 23.3M/23.3M [00:00<00:00, 38.4MB/s]


In [5]:
if not iskaggle:
    push_notebook('alexchalk', 'poisonous-mushrooms-submission-05',
                  title='Poisonous Mushrooms Submission 05',
                  file='02.ipynb',
                  competition=comp, private=True, gpu=False)

Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/alexchalk/poisonous-mushrooms-submission-05
